In [1]:
import os
import sys
from collections import defaultdict
from functools import reduce, partial

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


## OTUs table

In [2]:
_initiation = True

otus = pd.read_excel("../data/raw/baltika---ssu---otus.xlsx")
print(otus.shape)
otus.head()

(61848, 33)


,#sngs:otus#0000000038#0000001438,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,sample name,cluster id,cluster acc,# sequences,avg seq. ident.,similarity,sequence,reference,classifications,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,150cm Fin,125618,M70975:4:000000000-K68VJ:1:1101:2236:19011,1686,98.74,99.60,TACATAAGGAGCAAGCGTTATCCGGAATTATTGGGCGTAAAGGGCT...,JQ087016.1.1512,ncbi|138.1|244328|root,cellular organisms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,185cm Gotland,79360,M70975:4:000000000-K68VJ:1:1105:11330:14955,1672,98.96,98.50,GAGTGTCCGCCGCGGTAATACATAAGGAGCAAGCGTTATCCGGAAT...,JQ087016.1.1512,ncbi|138.1|244328|root,cellular organisms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
otus.columns = otus.loc[0]
otus.drop(0, inplace=True)
otus = otus.loc[~np.all(otus.isna(), axis=1)]

In [15]:
otus_main = otus[otus.columns[:9]]  #.to_csv("../data/")
otus_taxa = otus.iloc[: ,9:]
otus_main.head(2)

,sample name,cluster id,cluster acc,# sequences,avg seq. ident.,similarity,sequence,reference,classifications
1,150cm Fin,125618,M70975:4:000000000-K68VJ:1:1101:2236:19011,1686,98.74,99.60,TACATAAGGAGCAAGCGTTATCCGGAATTATTGGGCGTAAAGGGCT...,JQ087016.1.1512,ncbi|138.1|244328|root
2,185cm Gotland,79360,M70975:4:000000000-K68VJ:1:1105:11330:14955,1672,98.96,98.50,GAGTGTCCGCCGCGGTAATACATAAGGAGCAAGCGTTATCCGGAAT...,JQ087016.1.1512,ncbi|138.1|244328|root


In [6]:
otus_taxa.head(2)

,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cellular organisms,Bacteria <prokaryotes>,Terrabacteria group,Firmicutes,Clostridia,environmental samples <Clostridia>,uncultured Clostridia bacterium,\tsilva|138|44461|Bacteria,Caldatribacteriota,JS1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cellular organisms,Bacteria <prokaryotes>,Terrabacteria group,Firmicutes,Clostridia,environmental samples <Clostridia>,uncultured Clostridia bacterium,\tsilva|138|44461|Bacteria,Caldatribacteriota,JS1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
a = otus_taxa.iloc[:, 1:].fillna("").values.flatten()
print(len(a))
a = [x for x in a if isinstance(x, str)]
print(len(a))
print(set("".join(a)))
"&" in set("".join(a))

858199
858166
{'U', 'C', 'K', 'o', 's', 'I', 'L', '2', '/', 'P', 'j', 'm', '9', '+', '3', ')', 'E', '5', 'q', 'b', 'f', 'n', 'p', 'G', '>', 'X', ' ', '_', '[', 'T', '|', 'V', '\t', "'", 'd', '<', '0', 'g', ']', ':', 'M', '8', 'z', 'y', 'i', 'H', '.', 'h', '(', 'r', 'O', '-', 'D', 'l', 'c', 'A', 'W', 'Z', 'S', 'k', 'e', 'Y', 'a', '4', '7', 'R', '=', 'u', 'B', 'N', '6', 't', 'x', 'J', '1', 'w', 'F', 'Q', 'v'}


False

In [46]:
def to_str(ser):
    a = [x for x in ser.values if isinstance(x, str)]
    s = "&".join(a)
    return s

In [47]:
otus_main["cellularity"] = otus_taxa.iloc[:, 0]
otus_main["taxonomy"] = otus_taxa.iloc[:, 1:].apply(to_str, axis=1)

/tmp/ipykernel_24669/3715497962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otus_main["cellularity"] = otus_taxa.iloc[:, 0]
/tmp/ipykernel_24669/3715497962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otus_main["taxonomy"] = otus_taxa.iloc[:, 1:].apply(to_str, axis=1)


In [50]:
otus_main.rename(
    {
        "# sequences": "n_seq", 
        "avg seq. ident.": "avg_seq_ident", 
        "cluster acc": "cluster_acc",
        "sample name": "sample_name",
        "cluster id": "cluster_id",
    }, 
    axis=1, inplace=True
)

/tmp/ipykernel_24669/1023398022.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otus_main.rename(


In [58]:
otus_main.to_csv("../data/otus.csv", index=None)

In [59]:
otus_main_loaded = pd.read_csv("../data/otus.csv")
otus_main_loaded.head()

,sample_name,cluster_id,cluster_acc,n_seq,avg_seq_ident,similarity,sequence,reference,classifications,cellularity,taxonomy
0,150cm Fin,125618.0,M70975:4:000000000-K68VJ:1:1101:2236:19011,1686.0,98.74,99.60,TACATAAGGAGCAAGCGTTATCCGGAATTATTGGGCGTAAAGGGCT...,JQ087016.1.1512,ncbi|138.1|244328|root,cellular organisms,Bacteria <prokaryotes>&Terrabacteria group&Fir...
1,185cm Gotland,79360.0,M70975:4:000000000-K68VJ:1:1105:11330:14955,1672.0,98.96,98.50,GAGTGTCCGCCGCGGTAATACATAAGGAGCAAGCGTTATCCGGAAT...,JQ087016.1.1512,ncbi|138.1|244328|root,cellular organisms,Bacteria <prokaryotes>&Terrabacteria group&Fir...
2,80cm Fin,198092.0,M70975:4:000000000-K68VJ:1:2101:17935:5545,1001.0,98.95,100.00,TACATAAGGAGCAAGCGTTATCCGGAATTATTGGGCGTAAAGGGCT...,EU290732.1.1552,ncbi|138.1|77133|root,cellular organisms,Bacteria <prokaryotes>&environmental samples <...
3,145cm Gotland,72613.0,M70975:4:000000000-K68VJ:1:1102:16527:17352,968.0,99.46,100.00,TACATAAGGAGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGCT...,KM071712.1.1489,ncbi|138.1|77133|root,cellular organisms,Bacteria <prokaryotes>&environmental samples <...
4,80cm Fin,199229.0,M70975:4:000000000-K68VJ:1:2104:5841:9394,963.0,98.94,99.62,GAGCAGCCGCGGTAATACATAAGGAGCAAGCGTTATCCGGAATTAT...,AWNT01000010.42863.44412,ncbi|138.1|1383026|root,cellular organisms,Bacteria <prokaryotes>&unclassified Bacteria&B...


In [60]:
otus_main_loaded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37313 entries, 0 to 37312
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sample_name      36834 non-null  object 
 1   cluster_id       36834 non-null  float64
 2   cluster_acc      36834 non-null  object 
 3   n_seq            36834 non-null  float64
 4   avg_seq_ident    36834 non-null  float64
 5   similarity       36834 non-null  float64
 6   sequence         36834 non-null  object 
 7   reference        36834 non-null  object 
 8   classifications  36834 non-null  object 
 9   cellularity      36834 non-null  object 
 10  taxonomy         37313 non-null  object 
dtypes: float64(4), object(7)
memory usage: 3.1+ MB


## Chemical composition
manually

In [64]:
chem = pd.read_csv("../data/chem.csv")
chem

,Номер станции,"Глубина отбора, см",CH4 (ПИД),CH4 (ДТП-1),Водород,Гелий,CO2 (ДТП-2),N2 (ДТП-2),O2 (ДТП-2),C3H8 (ДТП-1),...,C4H6 (ПИД),i-C5H12 (ПИД),1-C4H8 (ПИД),C3H8 (ПИД),i-C4H10 (ПИД),C4H10 (ПИД),C3H4 (ПИД),C3H6 (ПИД),C2H4 (ПИД),C2H6 (ПИД)
0,2cm Gotland,0-5,0.008407,0.000000,0,67.367812,0.000000,42.844723,4.679873,0.000000,...,3.600000e-06,0.000029,5.210000e-06,0.000027,3.650000e-06,0.000021,2.270000e-06,1.190000e-05,0.000018,0.000026
1,20cm Gotland,15-20,0.067153,0.055450,0,79.674802,0.158838,54.796674,6.112924,0.000000,...,3.580000e-07,0.000046,1.250000e-07,0.000021,1.280000e-07,0.000007,1.000000e-06,5.480000e-07,0.000001,0.000003
2,40cm Gotland,30-40,0.121456,0.196695,0,74.013891,0.084205,28.636420,4.923303,0.000000,...,4.720000e-07,0.000014,8.770000e-07,0.000013,1.720000e-06,0.000004,3.390000e-06,7.080000e-06,0.000013,0.000011
3,80cm Gotland,60-70,0.216005,0.202217,0,68.833619,0.153139,31.461529,5.158495,0.000000,...,4.380000e-06,0.000002,2.300000e-06,0.000010,3.240000e-06,0.000006,5.050000e-06,5.820000e-06,0.000011,0.000013
4,185cm Gotland,160,5.466184,6.174498,0,80.786639,5.855488,20.283451,3.711208,0.000000,...,4.480000e-06,0.000004,2.480000e-06,0.000026,4.600000e-06,0.000015,6.180000e-06,3.180000e-06,0.000021,0.000058
5,220cm Gotland,260,4.883052,5.017994,0,80.031492,4.960687,21.809040,3.600742,0.000000,...,3.380000e-06,0.000031,1.490000e-06,0.000029,2.790000e-06,0.000013,3.930000e-06,6.910000e-06,0.000011,0.000027
6,270cm Gotland,290,4.341075,5.694717,0,66.129721,14.029773,37.678791,4.268271,0.000000,...,4.250000e-06,0.000042,1.850000e-06,0.000032,3.870000e-06,0.000018,5.450000e-06,8.620000e-06,0.000018,0.000059
7,2cm Fin,0-5,1.710656,2.298928,0,65.623789,5.082976,40.909482,4.503966,1.098452,...,4.190000e-06,0.000028,5.680000e-07,0.000039,2.510000e-06,0.000017,9.060000e-07,5.480000e-06,0.000011,0.000017
8,10cm Fin,5-10,0.073785,0.075458,0,82.301572,0.105186,23.888085,3.422590,0.000000,...,1.180000e-06,0.000041,1.940000e-06,0.000016,4.020000e-06,0.000006,4.470000e-06,7.620000e-06,0.000009,0.000011
9,20cm Fin,15-20,0.282331,0.300515,0,68.015030,0.810185,41.296865,4.316727,0.000000,...,1.020000e-07,0.000058,7.980000e-07,0.000035,9.350000e-07,0.000013,3.420000e-07,1.470000e-07,0.000013,0.000017


In [63]:
chem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Номер станции       24 non-null     object 
 1   Глубина отбора, см  24 non-null     object 
 2   CH4 (ПИД)           24 non-null     float64
 3   CH4 (ДТП-1)         24 non-null     float64
 4   Водород             24 non-null     int64  
 5   Гелий               24 non-null     float64
 6   CO2 (ДТП-2)         24 non-null     float64
 7   N2 (ДТП-2)          24 non-null     float64
 8   O2 (ДТП-2)          24 non-null     float64
 9   C3H8 (ДТП-1)        24 non-null     float64
 10  C2H6 (ДТП-1)        24 non-null     float64
 11  C4H10 (ДТП-1)       24 non-null     float64
 12  neo-C5H12 (ПИД)     24 non-null     float64
 13  n-C5H12 (ПИД)       24 non-null     float64
 14  C6H14 (ПИД)         24 non-null     float64
 15  C4H6 (ПИД)          24 non-null     float64
 16  i-C5H12 (П

## Samples composition

In [66]:
comp = pd.read_csv("../data/comp.csv")
comp

,2cm Gotland,20cm Gotland,40cm Gotland,80cm Gotland,120cm Gotland,145cm Gotland,185cm Gotland,220cm Gotland,270cm Gotland,2cm Fin,...,110cm Fin,150cm Fin,190cm Fin,190cm Fin.1,domain,phylum,class,order,family,genus
0,10.46120,1.1149,1.00465,0.54855,0.29410,0.91300,1.27650,0.89165,1.10160,2.25905,...,4.62990,2.93200,14.38855,0.8165,Bacteria,Actinobacteriota,Actinobacteria,Corynebacteriales,Mycobacteriaceae,Mycobacterium
1,3.50275,0.4323,0.23410,0.23175,0.11385,0.23995,0.38340,0.37140,0.16495,0.37970,...,0.43200,0.25725,1.48010,0.5103,Bacteria,Actinobacteriota,Thermoleophilia,Solirubrobacterales,Solirubrobacteraceae,Conexibacter
2,4.32900,15.8703,12.43845,16.66055,1.42445,2.88385,2.40605,16.13270,5.03310,3.04790,...,3.30370,2.81250,2.87730,1.2043,Bacteria,Chloroflexi,Anaerolineae,Anaerolineales,Anaerolineaceae,uncultured
3,3.19500,2.7759,1.50360,2.15655,0.79220,0.38665,0.31675,2.93155,0.42755,0.47560,...,0.72555,0.59005,1.31790,0.6940,Bacteria,Desulfobacterota,Desulfobacteria,Desulfobacterales,Desulfosarcinaceae,SEEP-SRB1
4,2.78275,1.7747,3.42070,0.80630,0.64980,1.71630,3.14345,1.94490,2.89585,1.75310,...,1.85370,4.98280,4.42205,6.5524,Bacteria,Actinobacteriota,Thermoleophilia,Solirubrobacterales,67-14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401,0.00000,0.0114,0.00000,0.00000,0.00000,0.00920,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.0000,Bacteria,Verrucomicrobiota,Verrucomicrobiae,S-BQ2-57 soil group,NaN,NaN
1402,0.00560,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.0000,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Verrucomicrobiales,Rubritaleaceae,Persicirhabdus
1403,0.00560,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.0000,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Verrucomicrobiales,Verrucomicrobiaceae,uncultured
1404,0.00000,0.0000,0.00000,0.00000,0.01850,0.00920,0.00790,0.01360,0.00000,0.01260,...,0.00750,0.00625,0.00000,0.0000,Bacteria,WS4,NaN,NaN,NaN,NaN


In [67]:
comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   2cm Gotland     1405 non-null   float64
 1   20cm Gotland    1406 non-null   float64
 2   40cm Gotland    1406 non-null   float64
 3   80cm Gotland    1406 non-null   float64
 4   120cm Gotland   1406 non-null   float64
 5   145cm Gotland   1406 non-null   float64
 6   185cm Gotland   1406 non-null   float64
 7   220cm Gotland   1406 non-null   float64
 8   270cm Gotland   1406 non-null   float64
 9   2cm Fin         1406 non-null   float64
 10  10cm Fin        1406 non-null   float64
 11  20cm Fin        1406 non-null   float64
 12  40cm Fin        1406 non-null   float64
 13  80cm Fin        1406 non-null   float64
 14  110cm Fin       1406 non-null   float64
 15  150cm Fin       1406 non-null   float64
 16  190cm Fin       1406 non-null   float64
 17  190cm Fin.1     1406 non-null   f